# 3.1 TensorFlow 计算模型——计算图

## 3.1.2 计算图的使用

In [21]:
import tensorflow as tf

a = tf.constant([1.0, 2.0], name="a")
b = tf.constant([2.0, 3.0], name="b")
result = a + b

sess = tf.Session()
sess.run(result)

array([ 3.,  5.], dtype=float32)

In [22]:
# 获取当前默认的计算图。
print(a.graph is tf.get_default_graph()) 

True


In [23]:
# 使用Graph生成新的计算图。不同计算图上的张量和运算不会共享
import tensorflow as tf

print(tf.__version__)
print(tf.__path__)

g1 = tf.Graph()
with g1.as_default():
    # 0.9.0版本可用，新版本报错：TypeError: __init__() got an unexpected keyword argument 'shape'
#     v = tf.get_variable("v", initializer=tf.zeros_initializer(shape=[1])) 
    v = tf.get_variable("v", initializer=tf.zeros_initializer()(shape=[1])) # 初始化为0
    
g2 = tf.Graph()
with g2.as_default():
    v= tf.get_variable("v", initializer=tf.ones_initializer()(shape=[1])) # 初始化为1

with tf.Session(graph=g1) as sess:
    tf.initialize_all_variables().run() 
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))
        
with tf.Session(graph=g2) as sess:
    # WARNING: use tf.global_variables_initializer instead
#     tf.initialize_all_variables().run()
    init = tf.global_variables_initializer()
    sess.run(init)
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))

1.4.1
['/usr/local/lib/python2.7/dist-packages/tensorflow']
[ 0.]
[ 1.]


In [24]:
# 指定运行设备
g = tf.Graph()
with g.device("/gpu:0"):
    result = a + b
    print(result)

Tensor("add_4:0", shape=(2,), dtype=float32)


## 3.2 TensorFlow 数据模型——张量

In [25]:
import tensorflow as tf
a = tf.constant([1.0, 2.0], name="a")
b = tf.constant([2.0, 3.0], name="b")
result = tf.add(a, b, name="result")
print(result) # 名字、维度、类型

Tensor("result:0", shape=(2,), dtype=float32)


In [32]:
# a = tf.constant([1, 2], name="a")
a = tf.constant([1, 2], name="a", dtype=tf.float32)
b = tf.constant([2.0, 3.0], name="b") 

# ValueError: Tensor conversion requested dtype int32 for Tensor with dtype float32:'Tensor("b_3:0", shape=(2,), dtype=float32)'
result = a + b

## 3.3 TensorFlow 运行模型——会话

In [29]:
sess = tf.Session()
with sess.as_default(): # 设置默认会话
    print(result.eval()) 

[ 3.  5.]


In [33]:
sess = tf.Session()
print(sess.run(result))
print(result.eval(session=sess)) # 指定会话

[ 3.  5.]
[ 3.  5.]


In [34]:
sess = tf.InteractiveSession() # 直接构建默认会话
print(result.eval())
sess.close()

[ 3.  5.]


In [ ]:
# 配置会话，设置不支持GPU时自动调整到CPU，日志中记录每个节点被安排了哪个设备
config = tf.ConfigProto(allow_soft_placement=True,log_device_placement=True)
sess1 = tf.InteractiveSession(config=config)
sess2 = tf.Session(config=config)